**Biomedical Software Engineering**

**Prof. Arthur Goldberg**

**Dept. Genetics and Genomic Sciences**

**Spring 1, 2020**

# `yield` and generators, *magic* methods

[`yield`](https://docs.python.org/3/reference/expressions.html#yield-expressions) is like return, but the run-time saves the state of the function which executes the `yield`.

In [0]:
def filtered_iteritems(d, filter_keys):
    """A generator that filters a dict's items to keys in `filter_keys`.

    Args:
        d (:obj:`dict`): dictionary to filter
        filter_keys (:obj:`list` of :obj:`str`): list of keys to retain

    Yields:
        :obj:`tuple`: (key, value) tuples from `d` whose keys are in `filter_keys`
    """
    # Note docstring uses Yields: instead of Returns:
    for key, val in d.items():
        if key not in filter_keys:
            continue
        yield key, val

presidents = {'Washington': 1,
              'Adams': 2,
              'Jefferson': 3,
              'Madison': 4}
# calling a generator (a function with yield) creates a generator object
filtered_iteritems(presidents, {'Adams'})

<generator object filtered_iteritems at 0x7f1b8ec81ca8>

In [0]:
filtered_presidents = filtered_iteritems(presidents, {'Washington', 'Jefferson'})
# a generator object is an iterator
# __next__() gets the next item in an iterator
for _ in range(2):
  print(filtered_presidents.__next__())

('Washington', 1)
('Jefferson', 3)


If no more calls to yield are provided by the generator, then `__next__()` raises a `StopIteration` exception.



In [0]:
filtered_presidents = filtered_iteritems(presidents, {'Washington', 'Jefferson'})
# a generator creates an iterator
# __next__() gets the next item in an iterator
for _ in range(3):
  print(filtered_presidents.__next__())

('Washington', 1)
('Jefferson', 3)


StopIteration: ignored

Generators are especially helpful for infinite sequences. (They don't fit in lists.)

Question for students: write a generator that creates a sequence of Fibonacci numbers

# *magic* methods for comparing objects

The rich comparison methods are documented in the [Basic customization](https://docs.python.org/3/reference/datamodel.html#basic-customization) section of the Data Model reference.

Each comparison operator maps to a method, as listed. E.g., `x<y` calls `x.__lt__(y)`, etc.

The example below shows part of the `Event` class in my [discrete event simulator, DE-Sim](https://github.com/KarrLab/de_sim). The simulator must execute events in non-decreasing time order. (Concurrent events are allowed).

To efficiently achieve this, `Events` are stored in a priority queue (provided by the [`heapq`](https://docs.python.org/3/library/heapq.html) package) which keeps the event with the lowest time at the start of the queue. 

In [0]:
class Event(object):
    """ An object that holds a discrete event simulation (DES) event

    Each DES event is scheduled by creating an `Event` instance and storing it in the
    simulator's event queue. To reduce interface errors the `message`
    attribute must be structured as specified in the `message_types` module.

    Attributes:
        creation_time (:obj:`float`): simulation time when the event is created (aka `send_time`)
        event_time (:obj:`float`): simulation time when the event must be executed (aka `receive_time`)
        sending_object (:obj:`SimulationObject`): reference to the object that sends the event
        receiving_object (:obj:`SimulationObject`): reference to the object that receives
            (aka executes) the event
        message (:obj:`SimulationMessage`): a `SimulationMessage` carried by the event; its type
            provides the simulation application's type for an `Event`; it may also carry a payload
            for the `Event` in its attributes.
    """

    def __init__(self, creation_time, event_time, sending_object, receiving_object, message):
        self.creation_time = creation_time
        self.event_time = event_time
        self.sending_object = sending_object
        self.receiving_object = receiving_object
        self.message = message

    def __lt__(self, other):
        """ Does this `Event` occur earlier than `other`?

        Args:
            other (:obj:`Event`): another `Event`

        Returns:
            :obj:`bool`: `True` if this `Event` occurs earlier than `other`
        """
        return self.event_time < other.event_time

    def __le__(self, other):
        """ Does this `Event` occur earlier or at the same time as `other`?

        Args:
            other (:obj:`Event`): another `Event`

        Returns:
            :obj:`bool`: `True` if this `Event` occurs earlier or at the same time as `other`
        """
        return not (other < self)

    def __gt__(self, other):
        """ Does this `Event` occur later than `other`?

        Args:
            other (:obj:`Event`): another `Event`

        Returns:
            :obj:`bool`: `True` if this `Event` occurs later than `other`
        """
        return self.event_time > other.event_time

    def __ge__(self, other):
        """ Does this `Event` occur later or at the same time as `other`?

        Args:
            other (:obj:`Event`): another `Event`

        Returns:
            :obj:`bool`: `True` if this `Event` occurs later or at the same time as `other`
        """
        return not (self < other)

def make_event(event_time):
    # For demo purposes, make an Event that contains only event_time
    # (self, creation_time, event_time, sending_object, receiving_object, message)
    return Event(None, event_time, None, None, None)

e2 = make_event(2)
e4 = make_event(4)
# try all rich comparison methods
assert e2 < e4
assert e4 > e2
assert e2 <= e4
assert e4 >= e2
# expect no exceptions

True

Let's use `Event` in a priority queue as it would be used by a discrete event simulator.

In [0]:
import heapq

event_queue = []
# initialze the event queue with some events
for et in [3, 7, 1, 11, 9]:
    heapq.heappush(event_queue, make_event(et))

def times(event_queue):
    return [e.event_time for e in event_queue]

print('initial queue', times(event_queue))

# Act like a simulator, repeatedly removing the smallest event and adding a new event.
# I'm being careful to avoid scheduling an event at a time that's earlier than
# an event that's already executed.
for new_et in [6, 7, 10, 20, 22]:
    next_event = heapq.heappop(event_queue)
    print('execute event at ', next_event.event_time)
    print('schedule event at ', new_et)
    heapq.heappush(event_queue, make_event(new_et))
    print('start of queue', event_queue[0].event_time)
print('final queue', times(event_queue))


initial queue [1, 7, 3, 11, 9]
execute event at  1
schedule event at  6
start of queue 3
execute event at  3
schedule event at  7
start of queue 6
execute event at  6
schedule event at  10
start of queue 7
execute event at  7
schedule event at  20
start of queue 7
execute event at  7
schedule event at  22
start of queue 9
final queue [9, 10, 20, 11, 22]


Questions for students: which rich comparison methods are missing from `Event`? What happens when you invoke the corresponding comparison operators? Why?
Why is it OK that these comparison methods are not implemented in `Event`?

In [0]:
e2 == e4

False

In [0]:
e2 == e2

True

In [0]:
e2 != e4

True

In [0]:
class F(object):
    pass

f1 = F()
f2 = F()
assert f1 == f1
assert f1 != f2
# no output expected